In [1]:
import numpy as np
import cv2

# 2

In [2]:
n_bins = 16
histogram = np.zeros([n_bins, n_bins, n_bins], dtype=np.float32)
histogram.shape

(16, 16, 16)

In [3]:
import imageio

paths = ['sunset.png', 'terrain.png']
images = [imageio.imread(p) for p in paths]
im = imageio.imread(paths[0]).astype(np.float32)/256
histogram = np.zeros([n_bins, n_bins, n_bins], dtype=np.float32)

### BEGIN SOLUTION
binned_im = (im * n_bins).astype(int)
#     height, width = im.shape[:2]
#     for i in range(height):
#         for j in range(width):
#             R_bin = binned_im[i, j, 0]
#             G_bin = binned_im[i, j, 1]ram
#             B_bin = binned_im[i, j, 2]
#             histogram[R_bin, G_bin, B_bin] += 1

np.add.at(histogram, (binned_im[..., 0], binned_im[..., 1], binned_im[..., 2]), 1)
histogram /= np.sum(histogram)

In [4]:
len(np.unique(histogram))

112

In [5]:
binned_im.shape

(225, 300, 3)

In [6]:
binned_im

Array([[[9, 7, 6],
        [9, 7, 6],
        [9, 7, 6],
        ...,
        [9, 7, 6],
        [9, 7, 6],
        [9, 7, 6]],

       [[9, 7, 6],
        [9, 7, 6],
        [9, 8, 6],
        ...,
        [9, 7, 6],
        [9, 7, 6],
        [9, 7, 6]],

       [[9, 7, 6],
        [9, 7, 6],
        [9, 7, 6],
        ...,
        [9, 7, 6],
        [9, 7, 6],
        [9, 7, 6]],

       ...,

       [[7, 5, 4],
        [7, 5, 4],
        [7, 5, 4],
        ...,
        [7, 5, 4],
        [7, 5, 4],
        [7, 5, 4]],

       [[7, 6, 4],
        [7, 6, 4],
        [7, 6, 4],
        ...,
        [7, 6, 4],
        [7, 5, 4],
        [7, 5, 4]],

       [[6, 6, 4],
        [7, 6, 4],
        [7, 6, 4],
        ...,
        [6, 5, 3],
        [6, 5, 3],
        [6, 5, 3]]])

In [7]:
data = im
r,g,b = np.meshgrid(*[np.linspace(0,1, dim) for dim in data.shape], indexing='ij')


In [8]:
r.shape

(225, 300, 3)

In [9]:
g.shape

(225, 300, 3)

In [10]:
[np.linspace(0,1, dim) for dim in data.shape]

[array([0.        , 0.00446429, 0.00892857, 0.01339286, 0.01785714,
        0.02232143, 0.02678571, 0.03125   , 0.03571429, 0.04017857,
        0.04464286, 0.04910714, 0.05357143, 0.05803571, 0.0625    ,
        0.06696429, 0.07142857, 0.07589286, 0.08035714, 0.08482143,
        0.08928571, 0.09375   , 0.09821429, 0.10267857, 0.10714286,
        0.11160714, 0.11607143, 0.12053571, 0.125     , 0.12946429,
        0.13392857, 0.13839286, 0.14285714, 0.14732143, 0.15178571,
        0.15625   , 0.16071429, 0.16517857, 0.16964286, 0.17410714,
        0.17857143, 0.18303571, 0.1875    , 0.19196429, 0.19642857,
        0.20089286, 0.20535714, 0.20982143, 0.21428571, 0.21875   ,
        0.22321429, 0.22767857, 0.23214286, 0.23660714, 0.24107143,
        0.24553571, 0.25      , 0.25446429, 0.25892857, 0.26339286,
        0.26785714, 0.27232143, 0.27678571, 0.28125   , 0.28571429,
        0.29017857, 0.29464286, 0.29910714, 0.30357143, 0.30803571,
        0.3125    , 0.31696429, 0.32142857, 0.32

In [11]:
# two arrays for x,y location , one array for channel
len([np.linspace(0,1, dim) for dim in data.shape])

3

In [12]:
colors = np.stack([r,g,b], axis=-1).reshape(-1, 3)
colors

array([[0. , 0. , 0. ],
       [0. , 0. , 0.5],
       [0. , 0. , 1. ],
       ...,
       [1. , 1. , 0. ],
       [1. , 1. , 0.5],
       [1. , 1. , 1. ]])

In [13]:
colors.shape

(202500, 3)

In [14]:
marker_sizes = 300 * data**(1/3)
marker_sizes.max(),"shape-->",marker_sizes.shape

(299.60886, 'shape-->', (225, 300, 3))

In [15]:
marker_sizes.flat

# 3

In [16]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import imageio
import cv2
import math
from scipy import ndimage
# AttrDict is an MIT-licensed library that provides mapping objects that 
# allow their elements to be accessed both as keys and as attributes
from attrdict import AttrDict
from mpl_toolkits.mplot3d import Axes3D

# Many useful functions
def plot_multiple(images, titles=None, colormap='gray',
                  max_columns=np.inf, imwidth=4, imheight=4, share_axes=False):
    """Plot multiple images as subplots on a grid."""
    if titles is None:
        titles = [''] *len(images)
    assert len(images) == len(titles)
    n_images = len(images)
    n_cols = min(max_columns, n_images)
    n_rows = int(np.ceil(n_images / n_cols))
    fig, axes = plt.subplots(
        n_rows, n_cols, figsize=(n_cols * imwidth, n_rows * imheight),
        squeeze=False, sharex=share_axes, sharey=share_axes)

    axes = axes.flat
    # Hide subplots without content
    for ax in axes[n_images:]:
        ax.axis('off')
        
    if not isinstance(colormap, (list,tuple)):
        colormaps = [colormap]*n_images
    else:
        colormaps = colormap

    for ax, image, title, cmap in zip(axes, images, titles, colormaps):
        ax.imshow(image, cmap=cmap)
        ax.set_title(title)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
    fig.tight_layout()
        
def load_image(f_name):
    return imageio.imread(f_name, as_gray=True).astype(np.float32)/255

def convolve_with_two(image, kernel1, kernel2):
    """Apply two filters, one after the other."""
    image = ndimage.convolve(image, kernel1)
    image = ndimage.convolve(image, kernel2)   
    return image

def gauss(x, sigma):
    return 1 / np.sqrt(2 * np.pi) / sigma * np.exp(- x**2 / 2 / sigma**2)

# derivative of Gaussian w.r.t x
def gaussdx(x, sigma):
    return (-1 / np.sqrt(2 * np.pi) / sigma**3 * x *
            np.exp(- x**2 / 2 / sigma**2))

def gauss_derivs(image, sigma):
    # The ceil of the scalar x is the smallest integer i, such that
    # np.ceil(-1.7)==-1 but np.floor(-1.7)==-2
    kernel_radius = np.ceil(3.0 * sigma)
    # Np.newaxis comes in very handy when you want to explicitly convert an 
    # 1D array to either a row vector or a column vector
    # numpy.newaxis is used to increase the dimension of the existing array by one more dimension
    # 1D array will become 2D array and # 2D array will become 3D array
    x = np.arange(-kernel_radius, kernel_radius + 1)[np.newaxis]
    G = gauss(x, sigma)
    D = gaussdx(x, sigma)
    image_dx = convolve_with_two(image, D, G.T)
    image_dy = convolve_with_two(image, G, D.T)
    return image_dx, image_dy

def gauss_filter(image, sigma):
    kernel_radius = np.ceil(3.0 * sigma)
    x = np.arange(-kernel_radius, kernel_radius + 1)[np.newaxis]
    G = gauss(x, sigma)
    return convolve_with_two(image, G, G.T)

def gauss_second_derivs(image, sigma):
    kernel_radius = np.ceil(3.0 * sigma)
    x = np.arange(-kernel_radius, kernel_radius + 1)[np.newaxis]
    G = gauss(x, sigma)
    D = gaussdx(x, sigma)
    
    image_dx, image_dy = gauss_derivs(image, sigma)
    image_dxx = convolve_with_two(image_dx, D, G.T)
    image_dyy = convolve_with_two(image_dy, G, D.T)
    image_dxy = convolve_with_two(image_dx, G, D.T)
    return image_dxx, image_dxy, image_dyy

def map_range(x, start, end):
    """Maps values `x` that are within the range [start, end) to the range [0, 1)
    Values smaller than `start` become 0, values larger than `end` become
    slightly smaller than 1."""
    # Given an interval, values outside the interval are clipped to the interval edges.
    # For example, if an interval of [0, 1] is specified, values smaller than 0 become 0,
    # and values larger than 1 become 1.
    return np.clip((x-start)/(end-start), 0, 1-1e-10)

# draw circle on the image on the specified point with an automatic selecting radius
def draw_keypoints(image, points):
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    radius = image.shape[1]//100+1
    for x, y in points:
        cv2.circle(image, (int(x), int(y)), radius, (1, 0, 0), thickness=2)        
    return image

# draw the lines that connects two features from two images which stuck together vertically im1|im2 
def draw_point_matches(im1, im2, point_matches):
    result = np.concatenate([im1, im2], axis=1)
    result = (result.astype(float)*0.6).astype(np.uint8)
    im1_width = im1.shape[1]
    for x1, y1, x2, y2 in point_matches:
        cv2.line(result, (x1, y1), (im1_width+x2, y2), 
                 color=(0,255,255), thickness=2, lineType=cv2.LINE_AA)
    return result


Bad key "text.kerning_factor" on line 4 in
/home/pk/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [17]:
"compute_gradient_histogram"
gray_im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY).astype(float)
dx, dy = gauss_derivs(gray_im, sigma=2.0)
gray_im.shape , dx.shape , dy.shape ,im.shape 

((225, 300), (225, 300), (225, 300), (225, 300, 3))

In [18]:
gradients = np.stack([dy, dx], axis=-1)
gradients.shape

(225, 300, 2)

In [19]:
histogram = np.zeros((n_bins,n_bins),dtype=np.float32)
histogram.shape

(16, 16)

In [20]:
histogram.reshape(-1).shape

(256,)

In [21]:
binned_im = (im * n_bins).astype(int)
np.add.at(histogram,(binned_im[:,0],binned_im[:,1]),1)

# 4

In [22]:
hist2 = np.array([[1,2],
                  [4,5]])
hist1 = np.array([[13,9],
                  [11,7]])
np.sum([[0, 1], 
        [0, 5]])

6

In [23]:
"all the operation here are elementwise"

'all the operation here are elementwise'

In [24]:
hist1-hist2

array([[12,  7],
       [ 7,  2]])

In [25]:
(hist1-hist2)**2

array([[144,  49],
       [ 49,   4]])

In [26]:
(hist1-hist2)**2/(hist1+hist2+.001)

array([[10.28497964,  4.45414053],
       [ 3.2664489 ,  0.33330556]])

# 5

In [27]:
dx, dy = gauss_derivs(gray_im, sigma=2.0)
gray_im.shape , dx.shape , dy.shape ,im.shape 

((225, 300), (225, 300), (225, 300), (225, 300, 3))

In [28]:
height , width = gray_im.shape
M = np.empty((height, width, 2, 2), dtype=float)
M.shape

(225, 300, 2, 2)

In [29]:
M[0,0,:,:]

array([[0., 0.],
       [0., 0.]])

In [30]:
M[:,:,0,1] = dx

In [31]:
M

array([[[[ 0.        ,  0.00023586],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.00065272],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.00094075],
         [ 0.        ,  0.        ]],

        ...,

        [[ 0.        , -0.00111087],
         [ 0.        ,  0.        ]],

        [[ 0.        , -0.00074235],
         [ 0.        ,  0.        ]],

        [[ 0.        , -0.0002592 ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.00023024],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.00063417],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.00090839],
         [ 0.        ,  0.        ]],

        ...,

        [[ 0.        , -0.00107937],
         [ 0.        ,  0.        ]],

        [[ 0.        , -0.00072019],
         [ 0.        ,  0.        ]],

        [[ 0.        , -0.00025269],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.00021461],
         [ 0.      

In [32]:
trace_M = np.trace(M, axis1=2, axis2=3)
trace_M.shape

(225, 300)

In [33]:
np.linalg.det(M).shape

(225, 300)

In [34]:
M[ : , : , 0, 0] = gauss_filter(dx * dx, 4)
M[ : , : , 0, 1] = gauss_filter(dx * dy, 4)
M[ : , : , 1, 0] = M[..., 0, 1]
M[ : , : , 1, 1] = gauss_filter(dy * dy, 4)

In [35]:
det_M = np.linalg.det(M)
# trace_M.shape == (height, width) , here we trace the 2*2 matrix for 
# each pixel
""" [[dx*dx,dx*dy],                 
     [dy*dx,dy*dy]]"""
trace_M = np.trace(M, axis1=2, axis2=3)

scores = det_M - 0.06 * np.square(trace_M)
scores.shape

(225, 300)

In [50]:
a = np.array([[1,2],
             [4,50]])
b = np.array([[13,9],
             [11,7]])
c = np.array([[16,14],
              [15,12]])
d = np.array([[19,8],
              [18,17]])
# the comparison of two matrix is elementwise
a < d 

array([[ True,  True],
       [ True, False]])

In [37]:
a[a < d] 

array([1, 2, 4])

In [38]:
# assign the elements of a to zero when they are smaller than
# corresponding elements of matrix d
a[a < d] = 0
a

array([[ 0,  0],
       [ 0, 50]])

In [39]:
np.pad(b, [(1, 1),(1, 1)], 'constant')

array([[ 0,  0,  0,  0],
       [ 0, 13,  9,  0],
       [ 0, 11,  7,  0],
       [ 0,  0,  0,  0]])

In [40]:
# the "np.pad" add pad of equal to zero to the border of matrix scores
scores_padded = np.pad(scores, [(1, 1),(1, 1)], 'constant')
h, w = scores_padded.shape
scores_out = scores.copy()
# 
for ox in [-1, 0, 1]:
    for oy in [-1, 0, 1]:
        neighbor_scores = scores_padded[1+oy:h-1+oy, 1+ox:w-1+ox]
        print(neighbor_scores.shape,scores_padded.shape,scores.shape,)
        scores_out[scores < neighbor_scores] = 0
        print(1+oy,h-1+oy, 1+ox,w-1+ox)

(225, 300) (227, 302) (225, 300)
0 225 0 300
(225, 300) (227, 302) (225, 300)
1 226 0 300
(225, 300) (227, 302) (225, 300)
2 227 0 300
(225, 300) (227, 302) (225, 300)
0 225 1 301
(225, 300) (227, 302) (225, 300)
1 226 1 301
(225, 300) (227, 302) (225, 300)
2 227 1 301
(225, 300) (227, 302) (225, 300)
0 225 2 302
(225, 300) (227, 302) (225, 300)
1 226 2 302
(225, 300) (227, 302) (225, 300)
2 227 2 302


In [41]:
score_threshold=1e-8
corner_ys, corner_xs = (scores > score_threshold).nonzero()

In [42]:
corner_ys.shape , corner_xs.shape , scores.shape

((3184,), (3184,), (225, 300))

In [43]:
225* 300

67500

In [44]:
np.stack([corner_xs, corner_ys], axis=1).shape

(3184, 2)

In [45]:
scores > score_threshold

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [46]:
x=(scores > score_threshold).nonzero()

In [47]:
x[0][0] , x[1]

(45, array([144, 145, 146, ..., 148, 149, 150]))

# 7

In [53]:
a = np.array([[1,2],
             [4,50]])

In [54]:
# for i, desc1 in enumerate(descriptors1)
# dist gives all the items of row i
for i, dist in enumerate(a):
    print(dist)

[1 2]
[ 4 50]


In [55]:
# descriptors2-desc1
# if an array has the same column size of a matrix then we can
# subtract/add it to the matrix and each rows of matrix will 
# subtracted/added by this array
a - np.array([9,11])

array([[-8, -9],
       [-5, 39]])

In [ ]:
# np.sum

In [58]:
a

array([[ 1,  2],
       [ 4, 50]])

In [56]:
np.sum(a)

57

In [57]:
np.sum(a,axis=0)

array([ 5, 52])

In [59]:
np.sum(a,axis=1)

array([ 3, 54])

In [60]:
# np.sum((descriptors2 - desc1)**2/(descriptors2 + desc1 + eps), axis=-1)
# sum over all columns, at the end it will be a vector with the same
# row size as before
np.sum(a,axis=-1)

array([ 3, 54])

In [70]:
d

array([[19,  8],
       [18, 17]])

In [65]:
#np.argmin
np.argmin(d,axis=1)

array([1, 1])

In [66]:
np.min(d, axis=1)

array([ 8, 17])

In [67]:
np.min(d, axis=0)

array([18,  8])

In [77]:
e = np.array([[15,8],
              [16,17]])
print(np.min(e, axis=0)) # Minima along the first axis
print(np.argmin(e, axis=0)) # Minima along the first axis


print(np.min(e, axis=1)) # Minima along the second axis
print(np.argmin(e, axis=1))

[15  8]
[0 0]
[ 8 16]
[1 0]


In [74]:
np.amin(a, axis=0)# Minima along the first axis

array([1, 2])

In [75]:
np.amin(a, axis=1)# Minima along the second axis

array([1, 4])

In [97]:
distances = np.array([[6,4,5],
                      [1,3,2],
                      [9,7,8]])

print(np.min(distances, axis=0)) # Minima along the first axis , in each column
print(np.argmin(distances, axis=0)) # Minima along the first axis

[1 3 2]
[1 1 1]


In [98]:
closest_ids = np.argmin(distances, axis=1)
closest_dists = np.min(distances, axis=1)
print(np.argmin(distances, axis=1))
print(np.min(distances, axis=1)) # Minima along the second axis, in each row


[1 0 1]
[4 1 7]


In [99]:
ids1 = np.argsort(closest_dists)
print(ids1)
ids2 = closest_ids[ids1]
print(ids2)


[1 0 2]
[0 1 1]


In [107]:
for i in range(3):
    # for each row shows the minimum member of it
    print(distances[i,closest_ids[i]],closest_ids[i])

4 1
1 0
7 1


In [108]:
for i in range(3):
    # for each row shows the minimum member of it
    print(distances[i,ids2],closest_ids[i])

[6 4 4] 1
[1 3 3] 0
[9 7 7] 1


# 8

numpy.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)[source]
Returns True if two arrays are element-wise equal within a tolerance.

In [116]:
# two arrays are not equal so the equality shows False
print([2.1,3.20001]==[2.1000004,3.2])
# we can compare two arrays with some tolerance
np.allclose([2.1,3.20001],[2.1000004,3.2])

False


True

numpy.linalg.svd(a, full_matrices=True, compute_uv=True, hermitian=False)

In [128]:
np.random.seed(42)
a = np.random.randn(9, 6)# + 1j*np.random.randn(9, 6)
b = np.random.randn(2, 7, 8, 3) #+ 1j*np.random.randn(2, 7, 8, 3)

In [132]:
a=np.round(a,3)
a

array([[ 0.497, -0.138,  0.648,  1.523, -0.234, -0.234],
       [ 1.579,  0.767, -0.469,  0.543, -0.463, -0.466],
       [ 0.242, -1.913, -1.725, -0.562, -1.013,  0.314],
       [-0.908, -1.412,  1.466, -0.226,  0.068, -1.425],
       [-0.544,  0.111, -1.151,  0.376, -0.601, -0.292],
       [-0.602,  1.852, -0.013, -1.058,  0.823, -1.221],
       [ 0.209, -1.96 , -1.328,  0.197,  0.738,  0.171],
       [-0.116, -0.301, -1.479, -0.72 , -0.461,  1.057],
       [ 0.344, -1.763,  0.324, -0.385, -0.677,  0.612]])

In [155]:
u, s, vh = np.linalg.svd(a, full_matrices=True)
u.shape, s.shape, vh.shape

((9, 9), (6,), (6, 6))

In [135]:
np.round(vh,3)

array([[-0.096,  0.793,  0.461,  0.044,  0.228, -0.308],
       [ 0.196,  0.581, -0.727, -0.116, -0.074,  0.276],
       [-0.617, -0.058, -0.186, -0.708,  0.229, -0.165],
       [ 0.044, -0.11 , -0.451,  0.328,  0.159, -0.806],
       [ 0.101,  0.083,  0.096, -0.317, -0.863, -0.358],
       [-0.748,  0.103, -0.11 ,  0.525, -0.346,  0.152]])

In [141]:
np.round(vh.T,3)

array([[-0.096,  0.196, -0.617,  0.044,  0.101, -0.748],
       [ 0.793,  0.581, -0.058, -0.11 ,  0.083,  0.103],
       [ 0.461, -0.727, -0.186, -0.451,  0.096, -0.11 ],
       [ 0.044, -0.116, -0.708,  0.328, -0.317,  0.525],
       [ 0.228, -0.074,  0.229,  0.159, -0.863, -0.346],
       [-0.308,  0.276, -0.165, -0.806, -0.358,  0.152]])

In [136]:
np.round(s,3)

array([4.533, 3.315, 2.703, 1.825, 1.526, 1.358])

In [137]:
np.round(u,3)

array([[ 0.05 , -0.205, -0.559,  0.217, -0.063,  0.286, -0.514,  0.482,
         0.112],
       [ 0.067,  0.283, -0.498,  0.371,  0.376, -0.498,  0.347,  0.105,
        -0.094],
       [-0.593,  0.126,  0.146,  0.219,  0.419, -0.063, -0.534, -0.248,
        -0.191],
       [ 0.019, -0.735,  0.289,  0.296,  0.297,  0.01 ,  0.234,  0.239,
        -0.297],
       [-0.093,  0.216,  0.069,  0.409,  0.228,  0.667,  0.345, -0.089,
         0.386],
       [ 0.45 ,  0.209,  0.52 ,  0.297,  0.101, -0.283, -0.324,  0.262,
         0.362],
       [-0.455, -0.049,  0.086,  0.475, -0.675, -0.249,  0.129,  0.079,
         0.131],
       [-0.303,  0.388,  0.219, -0.256,  0.046,  0.118,  0.158,  0.731,
        -0.261],
       [-0.362, -0.28 , -0.057, -0.364,  0.266, -0.257,  0.095,  0.143,
         0.699]])

In [142]:
vhT=vh.T

In [145]:
vhT*s

array([[-0.43589492,  0.64825285, -1.66913965,  0.08020863,  0.15439405,
        -1.01516342],
       [ 3.59639643,  1.92568305, -0.15732386, -0.20120958,  0.12735544,
         0.13950981],
       [ 2.08755973, -2.41167091, -0.50204125, -0.82224603,  0.14576739,
        -0.14890495],
       [ 0.19809814, -0.3835655 , -1.91323262,  0.59914467, -0.48307519,
         0.71295975],
       [ 1.03455354, -0.24371501,  0.6200855 ,  0.28927397, -1.31756555,
        -0.46912633],
       [-1.39821126,  0.91568609, -0.44611768, -1.47125178, -0.54624736,
         0.20689927]])

numpy.array_str(a, max_line_width=None, precision=None, suppress_small=None)

In [147]:
np.array_str(np.arange(3))

'[0 1 2]'

In [154]:
u1, s1, vh1 = np.linalg.svd(a)

In [156]:
u-u1

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [157]:
s1-s

array([0., 0., 0., 0., 0., 0.])

In [158]:
vh-vh1

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [163]:
point_matches = np.random.random((644,4))
n_matches = 644
A = np.empty((n_matches*2, 9))
for i, (x1, y1, x2, y2) in enumerate(point_matches):


    A[2*i] = [x1, y1, 1, 0, 0, 0, -x1*x2, -x2*y1, -x2]
    A[2*i+1] = [0, 0, 0, x1, y1, 1, -y2*x1, -y1*y2, -y2]
# YOUR CODE HERE

u, s, vh = np.linalg.svd(A)
#vt = vh.T
H = vh[:,-1]/vh[-1,-1]
u.shape ,s.shape , vh.shape

((1288, 1288), (9,), (9, 9))

In [164]:
s

array([38.59303005, 31.05978952, 10.2839282 ,  9.03482209,  7.30416874,
        7.26589773,  5.85002026,  2.35671584,  1.95850354])

In [165]:
vh

array([[-2.34319237e-01, -2.29013015e-01, -4.52051315e-01,
        -2.33520714e-01, -2.26819087e-01, -4.50839534e-01,
         2.55344225e-01,  2.52899616e-01,  4.99903137e-01],
       [ 2.97640282e-01,  2.88273443e-01,  5.71242087e-01,
        -2.98861498e-01, -2.89308290e-01, -5.73556738e-01,
         2.39145639e-04, -2.28619865e-03,  1.03223931e-03],
       [ 2.51200884e-01,  1.12541075e-01,  3.45819186e-01,
         2.41498847e-01,  1.52274565e-01,  3.28489978e-01,
         2.53497504e-01,  3.69614781e-01,  6.43700816e-01],
       [-3.29900265e-01,  4.38089106e-01,  2.11099158e-02,
        -3.39064679e-01,  4.49218697e-01,  2.21468068e-02,
         4.42082946e-01, -4.15768847e-01,  1.15083801e-01],
       [ 1.83316913e-01, -6.00509889e-01,  2.09683895e-01,
        -5.76552275e-01,  2.68442273e-01,  1.66143450e-01,
         2.27785419e-01,  1.86199820e-01, -2.07801469e-01],
       [-5.37020672e-01,  9.11184980e-02,  2.61662330e-01,
        -1.76859225e-02, -5.35583190e-01,  3.056787

In [ ]:
ab